In [ ]:
# snscrape --jsonl --max-results 10000 --since "2023-01-01" twitter-search 'from:"elonmusk" until:"2023-05-29"'> {}-tweets.json

In [1]:
import os
import pandas as pd
import json
def get_tweet_by_date(username,since_date,until_date,max_count):
    os.system("snscrape --jsonl --max-results {} --since {} twitter-search 'from:{} until:{}'> {}-tweets.json".format(max_count,since_date,username,until_date,username))
    # Reads the json generated from the CLI command above and creates a pandas dataframe
    tweets_df = pd.read_json('{}-tweets.json'.format(username), lines=True)
    DATASET_COLUMNS=['date', 'id', 'rawContent', 'retweetedTweet', 'quotedTweet', 'hashtags', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'viewCount']
    new_df = tweets_df[DATASET_COLUMNS].copy()
    return new_df

In [2]:
import nltk
import string
import re
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()
    text = text.replace('@', 'at')
    pat = r"[{}]".format(string.punctuation) 
    text = re.sub(pat, ' ', text) 
    text = re.sub('  +', ' ', text) 
    text = re.sub('https?://|www\.', '', text)  
    text = re.sub('\n', '', text)
    # text = re.sub('\w*\d\w*', '', text) 
    # print(text)
    # text = text.replace(u'\ud800', '')
    # text = text.replace('\x80', '?')
    # text = text.translate({i: None for i in range(128, 256)})
    # fixed_text = fix_nonascii(text)
    text = re.sub(r'[^\x00-\x7f]',r'',text)
    # text = fixed_text
    # print(text)
    
    # remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    if len(text) == 0:
        text = " "
    return text

In [ ]:
import os
import pandas as pd
import time
from datetime import datetime

username = "elonmusk"
since_date = "2023-01-01"
until_date = "2023-05-29"
max_count = 10000

# Get current date and time
now = datetime.now()

# Format current date as string
date_string = now.strftime("%Y-%m-%d")
print(date_string)  # Output: 2023-04-15

tweet_df = get_tweet_by_date(username,since_date,until_date,max_count)
print(tweet_df.shape)
print('Missing values counts')
print(tweet_df.isnull().sum())
# remove stop words
tweet_df['clean'] = tweet_df['rawContent'].apply(lambda row: clean_text(row))
# print(tweet_df.head())
tweet_df.to_csv('{}_tweets_29_{}.csv'.format(username,date_string), index=False)